In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time, traceback
import re
from urllib.parse import urlparse, parse_qs
from selenium.common.exceptions import (
    NoSuchWindowException, InvalidSessionIdException, WebDriverException, TimeoutException
)

## Paste this into terminal: /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \--remote-debugging-port=9222 \--user-data-dir="$HOME/chrome-debug-profile"

def extract_braze_campaign_data(url, data, timeout):
    
    # Cycles through campaigns based on campaigns per page
    for i in range(int(re.search(r"limit=(\d+)", url).group(1))):
        
        # Load the page of campaigns
        driver = safe_get(driver, url)
        time.sleep(timeout/2)
        driver.get(url)
        
        # Wait until the page loads the list of links
        link_elements = WebDriverWait(driver, timeout).until(
            lambda d: [el for el in d.find_elements(By.CSS_SELECTOR, "a.StyledLink-sc-1ytrr8d-0.gwHZaw.bcl-link")] 
                      if len(d.find_elements(By.CSS_SELECTOR, "a.StyledLink-sc-1ytrr8d-0.gwHZaw.bcl-link")) >= 10 
                      else False
        )
        
        # Filter to links that sent to users
        sliced_links = link_elements[1:-4]
        usable_links = [
            link for link in sliced_links
            if any(link.text.startswith(prefix) for prefix in ["App_", "RESEND_", "3RESEND_", "RESENDApp_", "Copy of App_"])
        ]

        # If no more useful links, return data
        if i >= len(usable_links):
            break

        # Click into desired campaign
        try:
            campaign = usable_links[i]
            name = campaign.text.strip()
            campaign.click()
            time.sleep(timeout/2)
            switch_to_last_window(driver)
            print(f"[CLICKING] Clicked into Campaign {i+1}: {name}")
        except:
            print(f"[WARNING] Could not click into campaign for: {name}")

        # Extract opens and sends
        try:
            wait = WebDriverWait(driver, timeout)
            
            and_sends_xpath = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div[2]/section[2]/div/div/div[1]/span[2]/div/table/thead/tr[3]/td[1]'
            ios_sends_xpath = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div[2]/section[2]/div/div/div[2]/span[2]/div/table/thead/tr[3]/td[1]'
            and_dir_opens_xpath = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div[2]/section[2]/div/div/div[1]/span[2]/div/table/thead/tr[3]/td[5]/div/div[1]/div[2]'
            ios_dir_opens_xpath = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div[2]/section[2]/div/div/div[2]/span[2]/div/table/thead/tr[3]/td[5]/div/div[1]/div[2]'

            and_sends = getRealNumber(and_sends_xpath, allow_fallback=False, default_if_missing=0)
            ios_sends = getRealNumber(ios_sends_xpath, allow_fallback=False, default_if_missing=0)
            and_dir_opens = getRealNumber(and_dir_opens_xpath, allow_fallback=False, default_if_missing=0)
            ios_dir_opens = getRealNumber(ios_dir_opens_xpath, allow_fallback=False, default_if_missing=0)

        except:
            print(f"[WARNING] Could not extract Push Rate Info for: {name}")
            and_sends = 'n/a'
            ios_sends = 'n/a'
            and_dir_opens = 'n/a'
            ios_dir_opens = 'n/a'

        
        # Click edit button
        try:
            edit_button = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable(
                    (By.XPATH, "//button[.//span[text()='Edit Campaign']]")
                )
            )
            edit_button.click()
            WebDriverWait(driver, timeout).until(
                lambda d: d.execute_script("return document.readyState") == "complete"
            )
        except:
            print(f"[WARNING] Could not find 'Edit Campaign' for: {name}")

        # Extract push title and message
        try:
            wait = WebDriverWait(driver, timeout)
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="main-col"]/div[2]/div[4]//section[2]'))
            )
            
            title_xpath = '//*[@id="main-col"]/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/div/div[2]/div/div/div[2]/span[1]'
            message_xpath = '//*[@id="main-col"]/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/div/div[2]/div/div/div[2]/span[2]'
        
            title_elem = wait_presence_with_recover(driver, (By.XPATH, title_xpath), timeout, url_when_recover=url)
            message_elem = wait_presence_with_recover(driver, (By.XPATH, message_xpath), timeout, url_when_recover=url)
        
            title = title_elem.text.strip()
            message = message_elem.text.strip()
        except:
            print(f"[WARNING] Could not extract Title and Message for: {name}")
            title = 'no title'
            message = 'no message'

        # Extract push image address (if it exists)
        try:
            canvas_xpath = "//*[@id='main-col']/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/div/div[2]/div/div/div[3]/div/div/div/canvas"
            image_elem = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.XPATH, canvas_xpath))
            )
            image_url = image_elem.get_attribute("src")
        except TimeoutException:
            image_url = 'no image'

        # Expand deeplink text box
        try:
            text_box_xpath = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/fieldset[4]/div[1]/div/div/div/div[2]/div/div[1]/div/div[1]/section/div/div'
            text_box_elem = driver.find_element(By.XPATH, text_box_xpath)
            driver.execute_script(
                "arguments[0].setAttribute('style', 'width: 232px; height:526px;');",
                text_box_elem
            )
        except:
            print(f"[WARNING] Could not enlarge deeplink text box.")
        
        # Extract UUID from deeplink
        try:
            deeplink_xpath_1 = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/fieldset[4]/div[1]/div/div/div/div[2]/div/div[1]/div/div[1]/section/div/div/div[1]/div[2]/div[1]/div[4]/div[3]/span/span'
            deeplink_xpath_2 = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/fieldset[4]/div[1]/div/div/div/div[2]/div/div[1]/div/div[1]/section/div/div/div[1]/div[2]/div[1]/div[4]/div[4]/span/span'
            deeplink_xpath_3 = '/html/body/div[4]/div[3]/div[2]/div/div[1]/div[2]/div[4]/div/div/div/div/div/div/div/div[2]/div/div/section[2]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/fieldset[4]/div[1]/div/div/div/div[2]/div/div[1]/div/div[1]/section/div/div/div[1]/div[2]/div[1]/div[4]/div[5]/span/span'
            
            deeplink_elem_1 = wait.until(EC.presence_of_element_located((By.XPATH, deeplink_xpath_1)))
            deeplink_elem_2 = wait.until(EC.presence_of_element_located((By.XPATH, deeplink_xpath_2)))
            deeplink_elem_3 = wait.until(EC.presence_of_element_located((By.XPATH, deeplink_xpath_3)))
        
            deeplink_1 = deeplink_elem_1.text.strip()
            deeplink_2 = deeplink_elem_2.text.strip()
            deeplink_3 = deeplink_elem_3.text.strip()

            deeplink = f"{deeplink_1}{deeplink_2}{deeplink_3}"
            uuid = re.search(r"[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{4}-[0-9a-fA-F]{12}", deeplink).group(0)
        except:
            uuid = 'no uuid'

        print("Total opens:", int(str(ios_dir_opens).replace(',', '')) + int(str(and_dir_opens).replace(',', '')))
        
        if len(message) < 1000 and (int(str(ios_dir_opens).replace(',', '')) + int(str(and_dir_opens).replace(',', ''))) >= 50:
            data.append({
                "Campaign Name": name,
                "Push Title": title,
                "Push Message": message,
                "Image URL": image_url,
                "UUID": uuid,
                "Andriod Sends": and_sends,
                "IOS Sends": ios_sends,
                "Andriod Direct Opens": and_dir_opens,
                "IOS Direct Opens": ios_dir_opens
            })

    return data

def getRealNumber(trigger_xpath, tooltip_timeout=8, allow_fallback=False, default_if_missing=0):
    """
    Hover/tooltip-first number extractor.
    - Returns an *int*.
    - If the element/tooltip isn't present (e.g., platform not used in this campaign), returns default_if_missing (0).
    - Handles '', '—', 'N/A', '1,234', '1.2K', '3M' robustly.
    """
    TOOLTIP_SEL = (
        ".bcl-Portal [role='tooltip'], "
        ".bcl-Portal .bcl-uncontrolled-tooltip, "
        ".bcl-Portal [class*='StyledTooltip'], "
        "[role='tooltip']"
    )

    def to_int(s, default=0):
        # Normalize a messy numeric string to an int
        if s is None:
            return default
        s = str(s).strip()
        if not s:
            return default

        # Remove spaces and commas for simple forms
        raw = s.replace(',', '').strip()

        # K/M suffixes (e.g., 1.2K, 3M)
        m = re.match(r'^([-+]?\d+(?:\.\d+)?)([kKmM])?$', raw)
        if m:
            num = float(m.group(1))
            suf = (m.group(2) or '').lower()
            mult = 1_000 if suf == 'k' else 1_000_000 if suf == 'm' else 1
            return int(num * mult)

        # Fallback: strip to digits/.- and try again
        raw2 = re.sub(r'[^\d\.\-]', '', raw)
        try:
            return int(float(raw2))
        except Exception:
            return default

    def nz(s): return " ".join((s or "").split())

    # ----- frame helpers -----
    def find_frame_containing(xpath):
        driver.switch_to.default_content()
        if driver.find_elements(By.XPATH, xpath):
            return None
        for f in driver.find_elements(By.TAG_NAME, "iframe"):
            try:
                driver.switch_to.frame(f)
                if driver.find_elements(By.XPATH, xpath):
                    return f
            except Exception:
                pass
            finally:
                driver.switch_to.default_content()
        return None

    def enter(frame):
        driver.switch_to.default_content()
        if frame is not None:
            driver.switch_to.frame(frame)

    def neutralize_tooltips():
        driver.switch_to.default_content()
        try:
            body = driver.find_element(By.TAG_NAME, "body")
            ActionChains(driver).move_to_element_with_offset(body, 1, 1).perform()
        except Exception:
            pass

    def js_hover(el):
        driver.execute_script("""
            const el = arguments[0];
            el.scrollIntoView({block:'center'});
            el.dispatchEvent(new MouseEvent('mouseover',  {bubbles:true}));
            el.dispatchEvent(new MouseEvent('mouseenter', {bubbles:true}));
            el.dispatchEvent(new MouseEvent('mousemove',  {bubbles:true}));
        """, el)

    def js_unhover(el):
        try:
            driver.execute_script("""
                const el = arguments[0];
                el.dispatchEvent(new MouseEvent('mouseleave', {bubbles:true}));
                el.dispatchEvent(new MouseEvent('mouseout',   {bubbles:true}));
            """, el)
        except Exception:
            pass

    def await_tooltip_text_topdoc(timeout_s):
        driver.switch_to.default_content()
        script = """
            const sel = arguments[0];
            const timeoutMs = arguments[1] * 1000;
            const cb = arguments[arguments.length - 1];
            const deadline = Date.now() + timeoutMs;

            function readTip(){
              const tips = Array.from(document.querySelectorAll(sel));
              const el = tips.at(-1);
              const text = el ? ((el.innerText || el.textContent || '').trim()) : '';
              return {count: tips.length, text};
            }
            const start = readTip();

            (function spin(){
               const now = readTip();
               if ((now.count > start.count) || (now.text && now.text !== start.text)) {
                  cb(now.text);
                  return;
               }
               if (Date.now() > deadline) {
                  cb(now.text || "");
                  return;
               }
               requestAnimationFrame(spin);
            })();
        """
        return driver.execute_async_script(script, TOOLTIP_SEL, float(timeout_s)) or ""

    def resolve_aria_describedby(el):
        ids = []
        try:
            v = (el.get_attribute("aria-describedby") or "").strip()
            if v: ids += v.split()
        except Exception:
            pass
        try:
            desc = el.find_elements(By.CSS_SELECTOR, "*[aria-describedby]")
            if desc:
                v2 = (desc[-1].get_attribute("aria-describedby") or "").strip()
                if v2: ids += v2.split()
        except Exception:
            pass
        ids = [i for i in dict.fromkeys(ids) if i]

        # same frame
        for i in ids:
            try:
                node = el.parent.execute_script("return document.getElementById(arguments[0])", i)
                if node:
                    txt = nz(node.text)
                    if txt: return txt
            except Exception:
                pass
        # top doc
        driver.switch_to.default_content()
        for i in ids:
            try:
                nodes = driver.find_elements(By.CSS_SELECTOR, f"#{i}")
                if nodes:
                    txt = nz(nodes[-1].text)
                    if txt: return txt
            except Exception:
                pass
        return ""

    # ----- locate element (tolerate missing platform) -----
    frame = find_frame_containing(trigger_xpath)
    enter(frame)
    nodes = driver.find_elements(By.XPATH, trigger_xpath)
    if not nodes:
        driver.switch_to.default_content()
        return int(default_if_missing)

    base = nodes[0]

    # ----- hover candidates -----
    candidates = [base]
    try:
        inside = base.find_elements(By.CSS_SELECTOR, "*[aria-describedby], *[data-tooltip], span:last-child, div:last-child")
        if inside:
            candidates.append(inside[-1])
    except Exception:
        pass

    for cand in candidates:
        try:
            neutralize_tooltips()
            enter(frame)
            js_hover(cand)

            aria_txt = resolve_aria_describedby(cand)
            if aria_txt:
                js_unhover(cand); driver.switch_to.default_content()
                return to_int(aria_txt, default_if_missing)

            tip_txt = await_tooltip_text_topdoc(tooltip_timeout)
            js_unhover(cand); driver.switch_to.default_content()
            if tip_txt:
                return to_int(tip_txt, default_if_missing)
        except Exception:
            try:
                js_unhover(cand)
            except Exception:
                pass
            driver.switch_to.default_content()

    # Fallbacks (attribute or visible text) if explicitly allowed
    if allow_fallback:
        enter(frame)
        for raw in [
            base.get_attribute("aria-label"),
            base.get_attribute("title"),
            base.get_attribute("data-tooltip"),
            base.text
        ]:
            raw = nz(raw)
            if raw:
                driver.switch_to.default_content()
                return to_int(raw, default_if_missing)

    driver.switch_to.default_content()
    return int(default_if_missing)

def spawn_driver():
    opts = Options()
    opts.add_experimental_option("debuggerAddress", DEBUGGER)
    # reduce tab discards / bfcache shenanigans
    opts.add_argument("--disable-features=TabFreeze,TabDiscarding,BackForwardCache")
    opts.add_argument("--remote-allow-origins=*")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--no-sandbox")
    return webdriver.Chrome(options=opts)

def ensure_driver(driver):
    """Recreate driver if session is dead."""
    try:
        _ = driver.current_window_handle
        return driver
    except (InvalidSessionIdException, WebDriverException, NoSuchWindowException):
        print("[RECOVER] Recreating Chrome driver")
        return spawn_driver()

def switch_to_last_window(driver):
    """Switch to the newest live window, if any."""
    try:
        handles = driver.window_handles
        if handles:
            driver.switch_to.window(handles[-1])
            return True
        return False
    except (NoSuchWindowException, WebDriverException):
        # try listing handles again; if none, report failure
        try:
            handles = driver.window_handles
            if handles:
                driver.switch_to.window(handles[-1])
                return True
        except Exception:
            return False
        return False

def safe_get(driver, url):
    """Navigate safely, recreating the driver if needed."""
    d = ensure_driver(driver)
    ok = switch_to_last_window(d)
    if not ok:
        # open a new window in this session
        try:
            d.execute_script("window.open('about:blank','_blank');")
            d.switch_to.window(d.window_handles[-1])
        except Exception:
            pass
    d.get(url)
    return d

def wait_presence_with_recover(driver, locator, timeout, url_when_recover=None):
    """
    presence_of_element_located with one recovery attempt if the window vanished.
    locator = (By.XPATH, "...") or (By.CSS_SELECTOR, "...")
    """
    try:
        return WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located(locator)
        )
    except (NoSuchWindowException, InvalidSessionIdException, WebDriverException):
        print("[RECOVER] Window lost during wait; reselecting window and retrying once")
        driver = ensure_driver(driver)
        if not switch_to_last_window(driver) and url_when_recover:
            driver.get(url_when_recover)
        return WebDriverWait(driver, max(3, int(timeout/2))).until(
            EC.presence_of_element_located(locator)
        )

DEBUGGER = "localhost:9222"  # same as manual Chrome launch

# ----- executable program -----

timeout = 10
url_limit = 100

# Loop pages by range (range * url limit = total checked campaigns)
for i in range(12): # 12 * 100 = 1,200 checked campaigns
    active_campaigns_url = f"https://dashboard-06.braze.com/engagement/campaigns/5fa2c479221359464225127a?start={i*url_limit}&limit={url_limit}&globalFilter=&columnFilters%5Bstatus%5D=stopped&sortby=last_edited&sortdir=-1&display=list"
    data = []
    df = pd.DataFrame(extract_braze_campaign_data(active_campaigns_url, data, timeout))
    df.to_csv(f"braze_campaign_updated_data_{i}.csv", index=False)
    print(f"[SAVED] {len(data)} rows saved to braze_campaign_updated_data_{i}.csv")

In [ ]:
import os
import numpy as np
import pandas as pd

# ---- config: keep your existing column names (including the "Andriod" typo) ----
AND_SENDS = "Andriod Sends"
IOS_SENDS = "IOS Sends"
AND_OPENS = "Andriod Direct Opens"
IOS_OPENS = "IOS Direct Opens"

# ---- vectorized parser for messy numeric Series: '', '—', '1,234', '1.2K', '3M' -> int ----
def parse_num_series(s: pd.Series) -> pd.Series:
    if s is None:
        return pd.Series([], dtype="int64")
    v = s.astype(str).str.strip().str.replace(',', '', regex=False).str.lower()
    # numeric part
    num = pd.to_numeric(v.str.extract(r'([-+]?\d*\.?\d+)')[0], errors='coerce')
    # suffix part
    suf = v.str.extract(r'([km])$')[0]
    mult = suf.map({'k': 1_000, 'm': 1_000_000}).fillna(1)
    out = (num * mult).fillna(0).astype('int64')
    return out

# ---- load CSVs (skip empties) ----
dfs = []
for i in range(11):
    path = f"braze_campaign_updated_data_{i}.csv"
    try:
        if not os.path.exists(path) or os.path.getsize(path) == 0:
            print(f"[SKIP EMPTY FILE] {path}")
            continue
        df_i = pd.read_csv(path)
        if df_i.shape[1] == 0:
            print(f"[SKIP NO-COLUMNS] {path}")
            continue
        dfs.append(df_i)
    except pd.errors.EmptyDataError:
        print(f"[SKIP EmptyDataError] {path}")
    except Exception as e:
        print(f"[SKIP] {path} ({e})")

if not dfs:
    raise RuntimeError("No usable CSVs to combine.")

# ---- combine, de-dup, minimal NA cleanup ----
all_data = (
    pd.concat(dfs, ignore_index=True)
      .drop_duplicates()
      .dropna(how="all")
)

# Ensure numeric columns exist (fill if missing so later steps don't KeyError)
for col in [AND_SENDS, IOS_SENDS, AND_OPENS, IOS_OPENS]:
    if col not in all_data.columns:
        all_data[col] = 0

# ---- make numeric versions (vectorized) ----
all_data["and_sends_num"] = parse_num_series(all_data[AND_SENDS])
all_data["ios_sends_num"] = parse_num_series(all_data[IOS_SENDS])
all_data["and_opens_num"] = parse_num_series(all_data[AND_OPENS])
all_data["ios_opens_num"] = parse_num_series(all_data[IOS_OPENS])

# Drop rows where *any* column equals 0 (your original intent) or string "0"
all_data = all_data.loc[~(all_data.eq(0)).any(axis=1)].copy()

# ---- Open Rates: conventionally opens / sends (guard divide-by-zero) ----
all_data["Android Open Rate"] = (
    all_data["and_opens_num"] / all_data["and_sends_num"].replace(0, np.nan)
)
all_data["IOS Open Rate"] = (
    all_data["ios_opens_num"] / all_data["ios_sends_num"].replace(0, np.nan)
)
all_data["Overall Open Rate"] = (
    (all_data["and_opens_num"] + all_data["ios_opens_num"]) /
    (all_data["and_sends_num"] + all_data["ios_sends_num"]).replace(0, np.nan)
)

# Replace NaNs from zero-division with 0
all_data[["Android Open Rate","IOS Open Rate","Overall Open Rate"]] = (
    all_data[["Android Open Rate","IOS Open Rate","Overall Open Rate"]]
      .fillna(0)
      .round(4)
)

# ---- Extract date and creator from Campaign Name ----
# Example suffix: ..._06.10.2025_Kyle
extract = all_data["Campaign Name"].astype(str).str.extract(r'(\d{2})\.(\d{2})\.(\d{4})_(\w+)')
extract.columns = ["month","day","year","Creator"]
all_data = pd.concat([all_data, extract], axis=1)

# Build Creation Date (YYYY-MM-DD)
all_data["Creation Date"] = (
    all_data["year"].fillna("").astype(str) + "-" +
    all_data["month"].fillna("").astype(str) + "-" +
    all_data["day"].fillna("").astype(str)
)

# Drop intermediate date parts if not needed
all_data = all_data.drop(columns=[c for c in ["month","day","year"] if c in all_data.columns])

# ---- Save ----
all_data.to_csv("braze_campaign_updated_data.csv", index=False)

print("All data saved to braze_campaign_updated_data.csv")